<a href="https://colab.research.google.com/github/mmzhang7/youtube_comments/blob/main/Datahacks_another.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import os
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
os.chdir('/content/drive/MyDrive/youtube')

In [ ]:
video_df = pd.read_csv('USvideos.csv')

In [ ]:
video_df

In [ ]:
comments_df = pd.read_csv('UScomments 2.csv', usecols=['video_id', 'comment_text', 'likes', 'replies'],encoding_errors='ignore')
comments_df['likes'] = pd.to_numeric(comments_df['likes'], errors='coerce')
comments_df['replies'] = pd.to_numeric(comments_df['replies'], errors='coerce')
comments_df = comments_df.dropna(subset=['likes', 'replies'])
comments_df = comments_df.drop_duplicates(subset=['video_id', 'comment_text'])
comments_df['comment_weight'] = comments_df['likes'] + comments_df['replies']

In [ ]:
comments_df

In [ ]:
with open('US_category_id.json', 'r') as file:
    data = file.read()
    categories = json.loads(data)


In [ ]:
comments_more_than_one_word = comments_df[comments_df['comment_text'].str.split().str.len() > 1]

In [ ]:
#merging two dfs on video_id (converted to string)
comments_more_than_one_word['video_id'] = comments_more_than_one_word['video_id'].astype(str)
video_df['video_id'] = video_df['video_id'].astype(str)
merged_df = pd.merge(comments_more_than_one_word, video_df, on = 'video_id', how = 'inner')
merged_df = merged_df.drop_duplicates(subset=['video_id', 'comment_text']).drop(columns = ['comment_total', 'views', 'date'])

In [ ]:
merged_df

In [ ]:
# !pip install fasttext
!pip install swifter
!wget https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.bin
!pip install langid

In [ ]:
#filter out non-English comments
!pip install langid
!pip install swifter
import langid
import swifter

def is_english_fast(text):
    if isinstance(text, str) and len(text.strip()) > 0:
        lang, _ = langid.classify(text)
        return lang == 'en'
    return False

merged_df['is_english'] = merged_df['comment_text'].swifter.apply(is_english_fast)
merged_english = merged_df[merged_df['is_english']]

In [ ]:
#merged_english = pd.merged_english.csv

In [ ]:
category_dict = {
    1: "Film & Animation",
    2: "Autos & Vehicles",
    10: "Music",
    15: "Pets & Animals",
    17: "Sports",
    18: "Short Movies",
    19: "Travels and Events",
    20: "Gaming",
    21: "Videoblogging",
    22: "People & Blogs",
    23: "Comedy",
    24: "Entertainment",
    25: "News & Politics",
    26: "Howto & Style",
    27: "Education",
    28: "Science & Technology",
    29: "Nonprofits & Activism",
    30: "Movies",
    31: "Anime/Animation",
    32: "Action/Adventure",
    33: "Classics",
    34: "Comedy",
    35: "Documentary",
    36: "Drama",
    37: "Family",
    38: "Foreign",
    39: "Horror",
    40: "Sci-Fi/Fantasy",
    41: "Thriller",
    42: "Shorts",
    43: "Shows",
    44: "Trailers"
}
merged_english['category_name'] = merged_english['category_id'].map(category_dict).drop(columns = ['category_id'])

In [ ]:
#convert category id to names, keep the top 10 most common
category_counts = merged_english['category_name'].value_counts()
top_categories = category_counts[category_counts >= 10000].head(10).index
merged_english_filtered = merged_english[merged_english['category_name'].isin(top_categories)]
merged_english_filtered

#clean comment texts like removing /n, links, non-punctuation symbols, etc.
import re
import string

def clean_text(text):
    text = text.lower()
    text = text.replace('\n', '')
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'@[A-Za-z0-9_]+', '', text)
    #text = re.sub(r'[^a-zA-Z0-9\s.,!?\'"]', '', text)
    text = re.sub(r'[^\w\s.,!?\'\"😃-🙏❤🧠🔥😭💀😅😉🤣🤡🤠😎🥲🥺🤔👀⭐🌟🌈🎉✨❤️🫶🤭😭❤️😍👍]+', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

merged_english_filtered['comment_text'] = merged_english_filtered['comment_text'].apply(clean_text)
merged_english_filtered

In [ ]:
#clean tags
merged_english['clean_tags'] = (
    merged_english['tags']
    .fillna('')
    .str.lower()
    .str.split('|')
    .apply(lambda x: list(dict.fromkeys([tag.strip() for tag in x if tag.strip()])))
)
merged_english

In [ ]:
from collections import Counter

tag_list = [tag for tag_list_row in merged_english['clean_tags'].dropna() for tag in tag_list_row]

tag_counts = Counter(tag_list)

excluded_tags = {'[none]', '', 'none'}

filtered_tag_counts = {tag: count for tag, count in tag_counts.items() if tag.lower() not in excluded_tags}

top_10_tags = Counter(filtered_tag_counts).most_common(10)

top_10_tags


In [ ]:
merged_english['comment_weight'].dtype

# Demo

In [ ]:
!pip install streamlit

In [ ]:
import streamlit as st

In [ ]:
st.title("Random Comment Generator with Models")

# Selectbox to choose a model
model_choice = st.selectbox("Choose a model", ["Markov Model 1", "Markov Model 2"])


## TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.pipeline import Pipeline

In [ ]:
tfidf_small = TfidfVectorizer(max_features=20000, ngram_range=(1,2))


In [ ]:
base_model_small = Pipeline([
    ('tfidf', tfidf_small),
    ('classifier', LogisticRegression(max_iter = 2000, class_weight = 'balanced'))
])

In [ ]:
weighted_model_small = Pipeline([
    ('tfidf', tfidf_small),
    ('classifier', LogisticRegression(max_iter = 2000))
])

In [ ]:
X = merged_english['comment_text']
y = merged_english['category_name']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

In [ ]:
base_model_small.fit(X_train, y_train)
y_pred_base = base_model_small.predict(X_test)
print("=== Baseline Model (Unweighted TF-IDF) ===")
print(classification_report(y_test, y_pred_base))

In [ ]:
weighted_model_small.fit(X_train, y_train, classifier__sample_weight=merged_english.loc[X_train.index, 'comment_weight'])
y_pred_weighted_base = weighted_model_small.predict(X_test)
print("\n=== Weighted Model ===")
print(classification_report(y_test, y_pred_weighted_base))

In [ ]:
models = {
    '5K': Pipeline([('tfidf', TfidfVectorizer(max_features=5000)), ('classifier', LogisticRegression(max_iter = 2000, class_weight = 'balanced'))]),
    '10K': Pipeline([('tfidf', TfidfVectorizer(max_features=10000)), ('classifier', LogisticRegression(max_iter = 2000, class_weight = 'balanced'))]),
    '20K': Pipeline([('tfidf', TfidfVectorizer(max_features=20000)), ('classifier', LogisticRegression(max_iter = 2000, class_weight = 'balanced'))])
}


for name, model in models.items():
    model.fit(X_train, y_train)
    score = model.score(X_test, y_test)
    print(f"{name} features: Accuracy = {score:.3f}")

## Markov Model

In [ ]:
!pip install markovify
!pip install transformers

In [ ]:
import markovify
import numpy as np
from transformers import pipeline, set_seed
music_comments = merged_english_filtered[merged_english_filtered['category_name'] == 'Music']['comment_text']
text_model = markovify.Text(' '.join(music_comments), state_size=4)

generator = pipeline('text-generation', model='gpt2')
set_seed(42)

prompt = "Category: Music | Mood: Hype | Comment:"
generator(prompt, max_length=50, num_return_sequences=5)

#keep comments with most 10 common tags
top_10_tags_set = set([tag for tag, _ in top_10_tags])
merged_english_filtered_on_both = merged_english_filtered[
    merged_english_filtered['clean_tags'].apply(lambda tag_list: any(tag in top_10_tags_set for tag in tag_list))
]
merged_english_filtered_on_both

#GPT-2 Transformer Generator using both category and tags
generator = pipeline('text-generation', model='gpt2')
set_seed(42)

def generate_youtube_comments(category, tags, mood="Hype", num_comments=5):
    prompt = f"Category: {category} | Tags: {tags} | Mood: {mood} | Comment:"

    outputs = generator(
        prompt,
        max_length=50,
        num_return_sequences=num_comments,
        do_sample=True,
        top_k=50,
        temperature=0.9
    )

    return [out['generated_text'] for out in outputs]

# Test example
category = "Music"
tags = "funny, vlog, humor"
mood = "Hype"

print("🎤 GPT-2 YouTube Comments for Music + Funny Tags:")
comments = generate_youtube_comments(category, tags, mood)

for i, comment in enumerate(comments, 1):
    print(f"{i}. {comment}")


## Trigram

In [ ]:
import nltk
from nltk import word_tokenize
from nltk.util import ngrams
nltk.download('punkt')
nltk.download('punkt_tab')

comments = merged_english_filtered.copy()
comments = comments['comment_text'].dropna().tolist()

tokens = []
for comment in comments:
    tokens.extend(word_tokenize(comment.lower()))
Trigrams = list(ngrams(tokens, 3))

In [ ]:
#generate with trigram
import collections
from collections import defaultdict, Counter
model = defaultdict(Counter)

for w1, w2, w3 in Trigrams:
    model[(w1, w2)][w3] += 1

def generate_comment(model, seed=("this", "video"), max_len=20):
    output = list(seed)
    for _ in range(max_len):
        next_words = model.get((output[-2], output[-1]), None)
        if not next_words:
            break
        next_word = random.choices(list(next_words.keys()), weights=next_words.values())[0]
        output.append(next_word)
    return " ".join(output)

In [ ]:
import random
print(generate_comment(model, seed=("i", "love")))
print(generate_comment(model, seed=("this", "video")))